In [57]:
import os
import pandas as pd
import torch
from ogb.graphproppred import PygGraphPropPredDataset
from torch_geometric.data import DataLoader
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder
from ogb.utils.features import (allowable_features, atom_to_feature_vector,
                                bond_to_feature_vector, atom_feature_vector_to_dict, bond_feature_vector_to_dict)
from rdkit import Chem
import numpy as np

# Extracting the Data using the OGB (Open Graph Benchmark) Library
The dataset we will be using is the OGBG-Molhiv dataset which is a molecular property prediction dataset adopted from MoleuleNet
1. Each Graph represents a molecule, where nodes are atoms, and edges are chemical bonds.
2. Input node features are 9-dimensional, containing atomic number, chirality, and formal charge and additional atom features.

In [20]:
data_directory = 'molecular_property_graph/'
sub_directory = 'ogbg_molhiv/'

In [12]:
# Download and process data at './dataset/ogbg_molhiv/'
dataset = PygGraphPropPredDataset(name = "ogbg-molhiv", root = data_directory)

Downloaded 0.00 GB: 100%|██████████| 3/3 [00:00<00:00,  3.85it/s]
Processing...


Extracting molecular_property_graph/hiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 41127/41127 [00:01<00:00, 30404.24it/s]


Converting graphs into PyG objects...


100%|██████████| 41127/41127 [00:01<00:00, 21630.41it/s]


Saving...


Done!


In [9]:
def get_directories(path):
    """
    Return a list of directory names within the specified path.
    """
    # List for storing directory names
    directories = []

    # List all files and directories in the specified path
    for entry in os.listdir(path):
        # Construct the full path
        full_path = os.path.join(path, entry)

        # Check if the entry is a directory
        if os.path.isdir(full_path):
            directories.append(entry)

    return directories

In [21]:
# Get the list of directories
graph_dir_list = get_directories(data_directory + sub_directory)

In [23]:
graph_dir_list

['split', 'mapping', 'processed', 'raw']

We notice that there are four main directories, namely: raw & split, which contain the csv data's in zip format. The other two directories are processed and processed_smiles, which contain the processed data in the form of pytorch geometric data objects.

# Data Preparation

## Unprocessed CSV Data

In [29]:
# Loading a sample csv data of graph labels & train data for investigation
# Path to the zip file
label_zip_path = data_directory + sub_directory + 'raw/graph-label.csv.gz'
train_zip_path = data_directory + sub_directory + 'split/scaffold/train.csv.gz'

In [32]:
graph_labels = pd.read_csv(label_zip_path, compression='gzip')
graph_train = pd.read_csv(train_zip_path, compression='gzip')

## Processed CSV Data (Graph)

In [61]:
# these functions taken from here: https://github.com/snap-stanford/ogb/blob/master/ogb/utils/mol.py
def ReorderCanonicalRankAtoms(mol):
    order = tuple(zip(*sorted([(j, i) for i, j in enumerate(Chem.CanonicalRankAtoms(mol))])))[1]
    mol_renum = Chem.RenumberAtoms(mol, order)
    return mol_renum, order

def smiles2graph(smiles_string, removeHs=True, reorder_atoms=False):
    """
    Converts SMILES string to graph Data object
    :input: SMILES string (str)
    :return: graph object
    """

    mol = Chem.MolFromSmiles(smiles_string)
    mol = mol if removeHs else Chem.AddHs(mol)
    if reorder_atoms:
        mol, _ = ReorderCanonicalRankAtoms(mol)

    # atoms
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom))
    x = np.array(atom_features_list, dtype = np.int64)

    # bonds
    num_bond_features = 3  # bond type, bond stereo, is_conjugated
    if len(mol.GetBonds()) > 0: # mol has bonds
        edges_list = []
        edge_features_list = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()

            edge_feature = bond_to_feature_vector(bond)

            # add edges in both directions
            edges_list.append((i, j))
            edge_features_list.append(edge_feature)
            edges_list.append((j, i))
            edge_features_list.append(edge_feature)

        # data.edge_index: Graph connectivity in COO format with shape [2, num_edges]
        edge_index = np.array(edges_list, dtype = np.int64).T

        # data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
        edge_attr = np.array(edge_features_list, dtype = np.int64)

    else:   # mol has no bonds
        edge_index = np.empty((2, 0), dtype = np.int64)
        edge_attr = np.empty((0, num_bond_features), dtype = np.int64)

    graph = dict()
    graph['edge_index'] = edge_index
    graph['edge_feat'] = edge_attr
    graph['node_feat'] = x
    graph['num_nodes'] = len(x)

    return graph


if __name__ == '__main__':
    graph = smiles2graph('O1C=C[C@H]([C@H]1O2)c3c2cc(OC)c4c3OC(=O)C5=C4CCC(=O)5')

We can see that node_feat has more columns than edge feat, which aligns with the description from the data source - where nodes are atoms with multiple features, and edges are chemical bonds with fewer features.

In [63]:
graph['node_feat'][:5, :]

array([[7, 0, 2, 5, 0, 0, 1, 0, 1],
       [5, 0, 3, 5, 1, 0, 1, 0, 1],
       [5, 0, 3, 5, 1, 0, 1, 0, 1],
       [5, 2, 4, 5, 1, 0, 2, 0, 1],
       [5, 1, 4, 5, 1, 0, 2, 0, 1]])

In [62]:
graph['edge_feat'][:5, :]

array([[0, 0, 1],
       [0, 0, 1],
       [1, 0, 1],
       [1, 0, 1],
       [0, 0, 0]])

## Feature Engineering (Encoding)

In [58]:
atom_encoder = AtomEncoder(emb_dim = 100)
bond_encoder = BondEncoder(emb_dim = 100)

atom_emb = atom_encoder(torch.from_numpy(graph['node_feat'])) # x is input atom feature
edge_emb = bond_encoder(torch.from_numpy(graph['edge_feat'])) # edge_attr is input edge feature


## Dataloaders for Pytorch Geometric Models

In [ ]:
# Using the torch data loader for preparation for a Machine Learning Model
split_idx = dataset.get_idx_split()
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False)

# Exploratory Data Analysis

## Unprocessed CSV Data

From the looks of the graph labels, it is a binary classification problem, where the label is 1  or 0, depending on whether the molecule is active or not.

In [39]:
graph_labels.value_counts()

0
0    39683
1     1443
Name: count, dtype: int64

From the looks of the describe function, the graph_train dataset has values that range in the same domain as the index, which is 0 to 41126. This may be a method by which the data is encoded and may be key to accessing the nodes. On its' own, it doesn't seem to be of much use.

In [46]:
graph_train.describe()

,3
count,32900.000000
mean,22999.375441
std,11683.650345
min,4.000000
25%,13688.750000
50%,24676.500000
75%,32901.250000
max,41126.000000


Printing the shape of the data shows us that the label and training data records do not share the same shape, and so this is telling us that they aren't meant to be used individually, as a machine learning model may not be able to process this data as-is as the labels will be unable to be paired with the training data due to differing dimensions. It may be more effective to investigate the processed data, that was stored in the processed folder as pytorch files and was used by the dataloaders to train the model.

In [38]:
print(graph_labels.shape)
print(graph_train.shape)

(41126, 1)
(32900, 1)


## Processed CSV Data (Graph)

Exploring the processed data, we can see that the node_feat and edge_feat are the features of the atoms and bonds respectively. The graph does not contain the column names due to the encoding process or for other reasons, and so we will have to refer to the data source to understand what each column represents. However, we do notice that the processed data has more information rich features with variables that have a more descriptive distribution than that of the unprocessed data.

In [67]:
atom_features = pd.DataFrame(graph['node_feat'])
atom_features.describe()

,0,1,2,3,4,5,6,7,8
count,23.000000,23.000000,23.000000,23.0,23.000000,23.0,23.000000,23.000000,23.000000
mean,5.521739,0.130435,2.869565,5.0,0.521739,0.0,1.217391,0.434783,0.826087
std,0.897956,0.457697,0.868873,0.0,0.845822,0.0,0.421741,0.506870,0.387553
min,5.000000,0.000000,1.000000,5.0,0.000000,0.0,1.000000,0.000000,0.000000
25%,5.000000,0.000000,2.500000,5.0,0.000000,0.0,1.000000,0.000000,1.000000
50%,5.000000,0.000000,3.000000,5.0,0.000000,0.0,1.000000,0.000000,1.000000
75%,6.000000,0.000000,3.000000,5.0,1.000000,0.0,1.000000,1.000000,1.000000
max,7.000000,2.000000,4.000000,5.0,3.000000,0.0,2.000000,1.000000,1.000000


In [68]:
chemicalbond_features = pd.DataFrame(graph['edge_feat'])
chemicalbond_features.describe()

,0,1,2
count,54.000000,54.0,54.000000
mean,1.333333,0.0,0.666667
std,1.427493,0.0,0.475831
min,0.000000,0.0,0.000000
25%,0.000000,0.0,0.000000
50%,1.000000,0.0,1.000000
75%,3.000000,0.0,1.000000
max,3.000000,0.0,1.000000
